# 要約 
このノートブックは、Kaggleの「LLM 20 Questions」コンペティションにおけるエージェント作成プロセスを示しています。具体的には、質問者と回答者の役割を果たすエージェントを構築し、ターゲットワードを推測するためのロジックを実装しています。

### 問題に取り組む内容
ノートブックでは、「20の質問」ゲームのエージェントを作成する過程が中心テーマです。これには、質問をする「質問者エージェント」と、「はい」または「いいえ」で回答する「回答者エージェント」を含む2つのエージェントを構築することが目的とされています。また、効率的な質問の生成と応答の解析を行い、ゲームを進行させることが求められています。

### 使用されている手法とライブラリ
- **PyTorch**: モデリングに使用される深層学習フレームワークです。エージェントはPyTorch上で動作するGemma言語モデルを利用しています。
- **Gemma**: Googleが提供する大規模言語モデルであり、特に因果推論と会話形式のプロンプトに対する応答生成に用いられます。
- **Forumatted conversation**: GemmaFormatterクラスを用いて、ユーザーとモデルのターンを構成するフォーマット機能を実装しています。このクラスは会話の状態を保持し、適切なプロンプトを生成します。
- **エージェントの設計**: `GemmaAgent`クラスとそのサブクラス（`GemmaQuestionerAgent`および`GemmaAnswererAgent`）が定義され、エージェントの初期化、プロンプト処理、応答生成などの機能が実装されています。
- **ユーティリティ関数**: 質問と応答を交互に組み合わせるための`interleave_unequal`関数や、エージェントを取得するための`get_agent`関数が用意されています。

### 生成されるアウトプット
ノートブックの実行により、`submission.tar.gz`というファイルが生成され、このファイルはKaggleのコンペティションに提出するために必要です。このプロセスにおいて、エージェントが大会の規則に従って機能するかどうかを確認するためのテストが行われます。

全体として、このノートブックは、言語モデルを活用してインタラクティブなゲーム環境を作成する方法を体系的に示しています。環境構築からエージェントの実装、最終的な提出準備までの一連の流れをカバーしています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連するが、初心者がつまずく可能性のある専門用語や概念を解説します。これにより、ノートブック特有のフレームワークや技術に対する理解を深める手助けとなることを目指しています。

1. **エージェント (Agent)**:
   - 機械学習や強化学習の文脈では、エージェントは環境と相互作用しながら行動を学習する主体を指します。このコンペティションでは、質問者（GemmaQuestionerAgent）と回答者（GemmaAnswererAgent）の2種類のエージェントが存在します。

2. **プロンプト (Prompt)**:
   - 言語モデルに対して、出力を生成するために与える入力文のことです。このノートブックでは、エージェントの振る舞いや特定のタスクを指示するために使用されます。

3. **`GemmaFormatter`**:
   - エージェントが生成する会話のフォーマットを担当するクラスです。ユーザーとモデルの対話を構造化し、ターンの開始と終了を示すトークンを使用して、整然とした会話形式に整えます。

4. **量子化 (Quantization)**:
   - モデルのパラメータを低いビット幅（例: 32ビット浮動小数点から8ビット整数）に変換するプロセスです。これにより、モデルのメモリ使用量を削減し、推論速度を向上させることができます。特にリソースが制限されている環境での使用がますます重要になっています。

5. **サンプリング (Sampling)**:
   - モデルからの出力を生成する際の手法のことです。「トップK」や「トップP」サンプリングなどのテクニックが用いられ、生成するトークンを選択する際のランダム性の度合いを調整します。これにより、生成されるテキストの多様性を制御します。

6. **コンテキストマネージャ (Context Manager)**:
   - Pythonにおける特別な構文で、特定のコードブロックの前後で特定の設定や状態を制御するために使用されます。本ノートブックでは、PyTorchのテンソルのデフォルトデータ型を一時的に設定するために使用されています。

7. **イテラブル (Iterable)**:
   - Pythonにおいて、ループ可能なオブジェクト（リスト、タプル、辞書など）を指します。このノートブックでは、特にターンを適用するためのデータ型を示す際に用いられています。

8. **正規表現 (Regular Expression)**:
   - 特定のパターンに基づいた文字列検索を行うための強力なツールです。このノートブックでは、応答から質問やキーワードを抽出する際に使用されています。

9. **自動微分 (Automatic Differentiation)**:
   - 深層学習のフレームワーク（この場合、PyTorch）で重要な特徴で、モデルのパラメータに関する勾配を自動的に計算する機能です。これは、最適化アルゴリズムが効率的に働くために必要です。

10. **チェックポイント (Checkpoint)**:
   - モデルの状態を保存するためのスナップショットです。トレーニング中に定期的にモデルの重みやバイアスを保存し、後で再開したり評価したりするために使用されます。

これらの用語は、専門用語の多い機械学習および深層学習の分野において、特に具体的な実装やフレームワークに関連しており、初心者が理解するのが難しいことがあります。

---


こんにちは皆さん、私はコードをいくつかの部分に分け、chatGPTから生成されたコメントや説明を追加しました。

このリンクのGemmaの例（https://www.kaggle.com/models/google/gemma/PyTorch/7b-it-quant/2）は、初心者にとって役立つかもしれません。

このノートブックが皆さんの役に立つことを願っています。

---

このノートブックは、**LLM 20 Questions**のエージェント作成プロセスを示しています。このノートブックを実行すると、`submission.tar.gz`ファイルが生成されます。このファイルは、右側の**コンペティションに提出**の見出しから直接提出できます。あるいは、ノートブックビューアから*Output*タブをクリックし、`submission.tar.gz`を見つけてダウンロードします。コンペティションのホームページの左上にある**エージェントを提出**をクリックして、ファイルをアップロードし、提出を行ってください。

In [ ]:
%%bash
cd /kaggle/working # 作業ディレクトリに移動します
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece # 必要なパッケージをインストールします
git clone https://github.com/google/gemma_pytorch.git > /dev/null # GitHubからgemma_pytorchリポジトリをクローンします（出力は表示しません）
mkdir /kaggle/working/submission/lib/gemma/ # gemma用のディレクトリを作成します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/ # gemmaのファイルを新しいディレクトリに移動します

# パート1: ファイル作成

- `%%writefile`は、ノートブックにこの行の下にあるすべてをファイルに書き込むよう指示します。
- `submission/main.py`は、ファイルが書き込まれるパスです。もし`submission`というディレクトリが存在しない場合は、作成されます。

In [ ]:
# #%%writefile submission/main.py
# # セットアップ
import os # オペレーティングシステムに関する機能を提供するモジュールをインポートします
import sys # Pythonのインタプリタに関する機能を提供するモジュールをインポートします

# パート2: 必要なライブラリのインポートとウェイトパスの設定

In [ ]:
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/" # Kaggleエージェントのパスを設定します
if os.path.exists(KAGGLE_AGENT_PATH): # Kaggleエージェントのパスが存在するか確認します
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib')) # 存在する場合、ライブラリパスをシステムパスに追加します
else:
    sys.path.insert(0, "/kaggle/working/submission/lib") # 存在しない場合、別のライブラリパスを追加します

import contextlib # コンテキストマネージャを使いやすくするモジュールをインポートします
import os # オペレーティングシステムに関する機能を提供するモジュールをインポートします
import sys # Pythonのインタプリタに関する機能を提供するモジュールをインポートします
from pathlib import Path # パス操作のためのモジュールをインポートします

import torch # PyTorchライブラリをインポートします
from gemma.config import get_config_for_7b, get_config_for_2b # Gemmaの設定用関数をインポートします
from gemma.model import GemmaForCausalLM # Gemmaのモデルをインポートします

if os.path.exists(KAGGLE_AGENT_PATH): # Kaggleエージェントのパスが存在するか確認します
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2") # 存在する場合、ウェイトパスを設定します
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2" # 存在しない場合、別のウェイトパスを設定します

# パート3: GemmaFormatterによるプロンプトフォーマット

- `GemmaFormatter`: ユーザーとモデルの間の会話をフォーマットするためのもので、ターンの開始と終了を示すために事前に定義されたトークンを使用します。

In [ ]:
import itertools # イテレータの操作を支援するモジュールをインポートします
from typing import Iterable # Iterableなデータ型を扱うための型ヒントをインポートします


class GemmaFormatter:
    _start_token = '<start_of_turn>' # ターンの開始を示すトークン
    _end_token = '<end_of_turn>' # ターンの終了を示すトークン

    # 初期化
    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt # システムプロンプトを設定します
        self._few_shot_examples = few_shot_examples # 少数の例を設定します
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n" # ユーザターンのフォーマット
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n" # モデルターンのフォーマット
        self.reset() # 状態をリセットします

    # 現在の会話の状態を文字列として返します
    def __repr__(self):
        return self._state

    # ユーザープロンプトを会話に追加するメソッド
    def user(self, prompt):
        self._state += self._turn_user.format(prompt) # プロンプトをユーザターンに追加します
        return self
        
    # モデルの応答を会話に追加するメソッド
    def model(self, prompt):
        self._state += self._turn_model.format(prompt) # プロンプトをモデルターンに追加します
        return self

    # ユーザターンの開始を示すメソッド
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n" # ユーザターンの開始マーカーを追加します
        return self

    # モデルターンの開始を示すメソッド
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n" # モデルターンの開始マーカーを追加します
        return self

    # 現在のターンの終了を示すメソッド
    def end_turn(self):
        self._state += f"{self._end_token}\n" # 終了マーカーを追加します
        return self

    # リセットメソッド
    def reset(self):
        # `_state`を空文字列で初期化します
        self._state = ""  

        # 提供されている場合、システムプロンプトを追加します。
        if self._system_prompt is not None:
            self.user(self._system_prompt)  
            
        # 提供されている場合、少数の例にターンを適用します
        if self._few_shot_examples is not None: 
            self.apply_turns(self._few_shot_examples, start_agent='user') # 初めのエージェントをユーザーに設定します
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        # 初めのエージェントに応じたフォーマッターの順序を設定します
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters) # フォーマッターをサイクルします
        for fmt, turn in zip(formatters, turns):
            fmt(turn) # 各ターンにフォーマッターを適用します
        return self

### 例

In [ ]:
# システムプロンプトと少数の例でフォーマッタを初期化します
formatter = GemmaFormatter(
    system_prompt="This is a system prompt.", # システムプロンプトを設定します
    few_shot_examples=["Example question?", "Example answer."] # 少数の例を設定します
)

# ユーザターンを追加します
formatter.user("What is the capital of France?") # 「フランスの首都は何ですか？」というユーザーの質問を追加します

# モデルターンを追加します
formatter.model("The capital of France is Paris.") # モデルの応答「フランスの首都はパリです」を追加します

# フォーマットされた会話を表示します
print(formatter) # 作成された会話の状態を出力します

`GemmaFormatter`クラスは、会話を一貫した方法で構造化しフォーマットするのに役立ち、ターンが適切にマークされ組織されることを保証します。

# パート4: エージェントの定義とユーティリティ

- `_set_default_tensor_type`コンテキストマネージャは、一時的にPyTorchテンソルのデフォルトデータ型を指定された型に設定し、コンテキストマネージャを使用するコードブロックが実行された後に元のtorch.floatにリセットします。

In [ ]:
import re # 正規表現操作のためのモジュールをインポートします

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """与えられたdtypeにデフォルトのtorchのdtypeを設定します。"""
    torch.set_default_dtype(dtype) # デフォルトのデータ型を指定された型に設定します
    yield # コンテキストブロック内のコードを実行します
    torch.set_default_dtype(torch.float) # 実行後、デフォルトのデータ型をtorch.floatにリセットします

### 例

In [ ]:
torch.tensor([1.0, 2.0]).dtype # 現在のデフォルトのデータ型でテンソルを作成し、そのデータ型を表示します

In [ ]:
with _set_default_tensor_type(torch.float64): # デフォルトのデータ型をtorch.float64に設定した状態で以下を実行します
    print(torch.tensor([1.0, 2.0]).dtype) # 指定されたデータ型でテンソルを作成し、そのデータ型を表示します

# パート5: 基本GemmaAgentクラス

GemmaAgentクラスは以下の目的で設計されています：

- 言語モデルを初期化し、設定する。
- プロンプトと応答をフォーマットし、処理する。
- コンテキストマネージャを使用してテンソルのデータ型を一時的に設定する。
- フォーマットされたプロンプトに基づいて応答を生成するためにモデルと対話する。

In [ ]:
class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant # モデルのバリアントを設定します
        self._device = torch.device(device) # 使用するデバイス（GPUまたはCPU）を設定します
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples) # フォーマッタを初期化します

        print("モデルを初期化しています")
        
        # モデルの設定
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b() # バリアントに応じてモデルの設定を取得します
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model") # トークナイザーのパスを設定します
        model_config.quant = "quant" in variant # 量子化設定を行います
        
        # モデルの初期化
        with _set_default_tensor_type(model_config.get_dtype()): # モデルのデフォルトデータ型を設定します
            model = GemmaForCausalLM(model_config) # 言語モデルを初期化します
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt') # チェックポイントのパスを設定します
            model.load_weights(ckpt_path) # チェックポイントからモデルの重みをロードします
            self.model = model.to(self._device).eval() # モデルをデバイスに移動し、評価モードにします

    def __call__(self, obs, *args):
        self._start_session(obs) # セッションを開始します
        prompt = str(self.formatter) # フォーマットされたプロンプトを取得します
        response = self._call_llm(prompt) # モデルにプロンプトを渡して応答を生成します
        response = self._parse_response(response, obs) # 応答を解析します
        print(f"{response=}") # 生成された応答を表示します
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError # サブクラスで実装する必要があります

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = { # サンプラーの引数のデフォルト値を設定します
                'temperature': 0.01, # 温度パラメータ
                'top_p': 0.1, # トップPサンプリング
                'top_k': 1, # トップKサンプリング
        }
        response = self.model.generate( # モデルに応じた新しいトークンを生成します
            prompt,
            device=self._device, # 使用するデバイスを指定します
            output_len=max_new_tokens, # 生成するトークン数を指定します
            **sampler_kwargs, # 他のサンプラー引数を渡します
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response) # 応答からキーワードを解析します
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower() # キーワードを小文字に変換します
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError # サブクラスで実装する必要があります

# パート6: GemmaQuestionerAgentクラス

In [ ]:
def interleave_unequal(x, y):
    # 2つのリスト（xとy）を交互に組み合わせ、片方が短い場合は残りを無視します
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]

class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # 親クラスの初期化を行います

    def _start_session(self, obs):
        self.formatter.reset() # フォーマッタをリセットします
        self.formatter.user("20の質問を始めましょう。あなたは質問者の役割を演じています。") # ユーザーターンを追加します
        turns = interleave_unequal(obs.questions, obs.answers) # 質問と回答を交互に組み合わせます
        self.formatter.apply_turns(turns, start_agent='model') # フォーマッタにターンを適用します
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えられる質問をしてください。") # 質問を促します
        elif obs.turnType == 'guess':
            self.formatter.user("今度はキーワードを推測してください。推測は二重アスタリスクで囲んでください。") # 推測を促します
        self.formatter.start_model_turn() # モデルターンを開始します

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', '')) # 応答から質問を解析します
            if match is None:
                question = "それは人ですか？" # デフォルトの質問を設定します
            else:
                question = match.group() # 見つかった質問を取得します
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response) # 推測されたキーワードを解析します
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType) # 不正なターンタイプに対するエラーを発生させます

GemmaQuestionerAgent:
- `__init__` ： 親クラスのコンストラクタを呼び出してエージェントを設定します。
- `_start_session` : 質問と回答を交互に組み合わせ、会話のフォーマットを設定します。
- `_parse_response` : エージェントが質問をしているか推測をしているかによって、モデルの応答を異なる方法で解釈します。

# パート7: GemmaAnswererAgentクラス

In [ ]:
class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # 親クラスの初期化を行います

    def _start_session(self, obs):
        self.formatter.reset() # フォーマッタをリセットします
        self.formatter.user(f"20の質問を始めましょう。あなたは回答者の役割を演じています。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。") # ユーザーターンを追加します
        turns = interleave_unequal(obs.questions, obs.answers) # 質問と回答を交互に組み合わせます
        self.formatter.apply_turns(turns, start_agent='user') # フォーマッタにターンを適用します
        self.formatter.user(f"この質問はキーワード{obs.keyword}に関するもので、カテゴリは{obs.category}です。はいかいいえで答えてください。答えは二重アスタリスクで囲んでください（例: **yes** または **no**）。") # 答えの形式を指示します
        self.formatter.start_model_turn() # モデルターンを開始します

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response) # 応答からキーワードを解析します
        return 'yes' if 'yes' in answer else 'no' # 応答に「yes」が含まれていれば'yes'を、そうでなければ'no'を返します

# パート8: エージェントの作成と関数定義

In [ ]:
# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者のはい・いいえの質問に応じて答えます。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問を始めましょう。あなたは質問者の役割を演じています。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** さて、キーワードを推測してください。",
    "**フランス**", "正解！",
]

# **重要:** エージェントをグローバルに定義して、必要なエージェントだけをロードします。
# 両方をロードすると、アウトオブメモリ（OOM）になる可能性があります。

# エージェント変数を初期化します
agent = None

# 名前に基づいて適切なエージェントを取得する関数
def get_agent(name: str):
    global agent
    
    # エージェントが初期化されておらず、要求されたエージェントが「質問者」である場合
    if agent is None and name == 'questioner':
        # 特定のパラメータでGemmaQuestionerAgentを初期化します
        agent = GemmaQuestionerAgent(
            device='cuda:0',  # 計算用デバイス
            system_prompt=system_prompt,  # エージェントのシステムプロンプト
            few_shot_examples=few_shot_examples,  # エージェントの行動をガイドする例
        )
    # エージェントが初期化されておらず、要求されたエージェントが「回答者」である場合
    elif agent is None and name == 'answerer':
        # 同じパラメータでGemmaAnswererAgentを初期化します
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    
    # エージェントが初期化されていることを確認します
    assert agent is not None, "エージェントが初期化されていません。"

    # 初期化されたエージェントを返します
    return agent

# 観測に基づいて相互作用を処理する関数
def agent_fn(obs, cfg):
    # 観測が質問をするためのものである場合
    if obs.turnType == "ask":
        # 「質問者」エージェントを取得して観測に応答します
        response = get_agent('questioner')(obs)
    # 観測が推測をするためのものである場合
    elif obs.turnType == "guess":
        # 「質問者」エージェントを取得して観測に応答します
        response = get_agent('questioner')(obs)
    # 観測が回答を提供するためのものである場合
    elif obs.turnType == "answer":
        # 「回答者」エージェントを取得して観測に応答します
        response = get_agent('answerer')(obs)
    
    # エージェントからの応答がNoneまたは非常に短い場合
    if response is None or len(response) <= 1:
        # ポジティブな応答（「はい」）と仮定します
        return "はい"
    else:
        # エージェントから受け取った応答を返します
        return response

1. **GemmaFormatterクラス**: このクラスは、ゲームのプロンプトをフォーマットする役割を担っています。ユーザーとモデルのターンを構成し、ユーザーとモデルのターンを開始し、ターンを終了し、状態をリセットし、ターンを適用するメソッドを持っています。エージェントに対するプロンプトの一貫したフォーマットを保証します。

2. **GemmaAgentクラス**: ゲームにおける一般的なエージェントを表す抽象クラスです。初期化、呼び出し、セッションの開始、言語モデル（LLM）の呼び出し、応答の解析、デフォルトテンソルタイプの設定といった共通のメソッドと属性を定義しています。

3. **GemmaQuestionerAgentクラス**と**GemmaAnswererAgentクラス**: これらのクラスはGemmaAgentから継承され、質問者と回答者のエージェントの特定の動作を実装しています。エージェントの動作をカスタマイズするために、`_start_session`および`_parse_response`メソッドをオーバーライドしています。

4. **interleave_unequal関数**: これは異なる長さの2つのリストを交互に組み合わせる関数です。ゲーム内で質問と回答を交互に組み合わせるために使用されます。

5. **get_agent関数**: この関数は、入力された名前（「質問者」または「回答者」）に基づいて適切なエージェントを初期化して返します。エージェントのインスタンスが1つだけ作成され再利用されることを保証します。

6. **agent_fn関数**: この関数はゲームのエントリーポイントとして機能します。観測のターンタイプ（「ask」、「guess」、「answer」）に基づいて使用するエージェントのタイプを決定し、対応するエージェントの`__call__`メソッドを呼び出して応答を生成します。

In [ ]:
# !apt install pigz pv > /dev/null # pigzとpvをインストールします（出力は表示しません）

In [ ]:
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/gemma/pytorch/7b-it-quant/2 
# submissionディレクトリと指定のgemmaモデルパスを圧縮してsubmission.tar.gzファイルを作成します。圧縮にはpigzを使用し、進行状況をpvで表示します。

---

# コメント

> ## Mohamed MZAOUALI
> 
> 今はずっと理解できるようになりました。ありがとうございます！
> 
> 

---